In [2]:
import pandas as pd
import numpy as np

import urllib3
import requests
from PIL import Image

import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud, STOPWORDS
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import classification_report, precision_score, recall_score, f1_score, accuracy_score
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC
from sklearn import metrics
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn import preprocessing
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from nltk.stem.porter import *
from nltk import word_tokenize
from nltk.stem.snowball import SnowballStemmer
from collections import Counter
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
from sklearn.metrics import accuracy_score
import nltk

nltk.download('punkt')

%matplotlib inline

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [3]:
train_data = pd.read_csv('datasets/q1/train.csv', sep=',')
test_data = pd.read_csv('datasets/q1/test_without_labels.csv', sep=',')


my_additional_stop_words = ['said','still','day','will','new','may','two','one','now','time','say','second','month','first','going','year','back','people','case','according']
stop_words = STOPWORDS.union(my_additional_stop_words)

In [6]:
nltk.download('stopwords')
stopWords = stopwords.words('english')
import re
stopWords.extend(['saying', 'said', 'say', 'yes', 'instead', 'meanwhile', 'right', 'really', 'finally', 'now', 
                       'one', 'suggested', 'says', 'added', 'think', 'know', 'though', 'let', 'going', 'back',
                       'well', 'example', 'us', 'yet', 'perhaps', 'actually', 'oh', 'year', 'lastyear',
                       'last', 'old', 'first', 'good', 'maybe', 'ask', '.', ',', ':', 'take', 'made', 'n\'t', 'go', 
                       'make', 'two', 'got', 'took', 'want', 'much', 'may', 'never', 'second', 'could', 'still', 'get', 
                       '?', 'would', '(', '\'', ')', '``', '/', "''", '%', '#', '!', 'next', "'s", ';', '[', ']', '...',
                       'might', "'m", "'d", 'also', 'something', 'even', 'new', 'lot', 'a', 'thing', 'time', 'way',
                       'always', 'whose', 'need', 'people', 'come', 'become', 'another', 'many', 'must', 'too', 'as', 'well'])

stopWords.extend([' ','are','and','I','A','And','So','arnt','This','When','It','many','Many','so','cant','Yes','yes','No','no','These','these'])

def processText(text):    
    p_stemmer = SnowballStemmer(language='english')
    text = re.sub(r'[^\w\s]','',text)
    tokens = text.lower()
    tokens = tokens.split()
    #tokens = [wrd for wrd in tokens if(wrd not in stopWords)]
    #lemmatizer = WordNetLemmatizer()
    #tokens = [lemmatizer.lemmatize(t) for t in tokens]
    #tokens = [wrd for wrd in tokens if(wrd not in stopWords)]
    stems = [p_stemmer.stem(t) for t in tokens]
    stems = [wrd for wrd in stems if(wrd not in stopWords)]
    stems = ' '.join(stems)
    return stems


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [7]:
print("building Hashing vectorizer")
hvect = HashingVectorizer()

print("building tfidf")
vectorizer = TfidfVectorizer(use_idf=True)
#vectorizer = TfidfVectorizer(use_idf=True)

building Hashing vectorizer
building tfidf


In [9]:
df = train_data['Title'] + train_data['Title'] + train_data['Content'] + train_data['Title'] + train_data['Title']
df = df.map(processText)

In [12]:
from sklearn.linear_model import SGDClassifier
sgd = SGDClassifier()

tfidfv = TfidfTransformer()

le = LabelEncoder()

X = np.array(df).flatten()
y = le.fit_transform(train_data['Label'])

X = np.array(X).flatten()
y = np.array(y).flatten()

In [19]:
accuracy,precision,recall,f1 = 0,0,0,0
kf = KFold(n_splits=5)
from numpy import unique

for trainI,testI in kf.split(X):
    #kfTestCategIds = le.transform(y[testI])
    sgd.partial_fit(tfidfv.fit_transform(hvect.fit_transform(X[trainI])),y[trainI],classes=unique(y))
    predictions = sgd.predict(tfidfv.fit_transform(hvect.fit_transform(X[testI])))
    
    ret = precision_score(y[testI], predictions, average='macro')
    precision += ret

    ret = recall_score(y[testI], predictions, average='macro')
    recall += ret

    ret = f1_score(y[testI], predictions, average='macro')
    f1 += ret

    accuracy += accuracy_score(y[testI], predictions)
    
#accuracy,precision,recall,f1 = accuracy/float(5),precision/float(5),recall/float(5),f1/float(5)
accuracy,precision,recall,f1 = accuracy/5,precision/5,recall/5,f1/5
print("Accuracy = ",end=' ')
print(accuracy)
print("Precision = ",end=' ')
print(precision)
print("Recall = ",end=' ')
print(recall)
print("f1-score = ",end=' ')
print(f1)

Accuracy =  0.9663491211592646
Precision =  0.9658469974160344
Recall =  0.9610289978564847
f1-score =  0.9633642354674988


In [20]:
df_test = test_data['Title']+test_data['Title']+test_data['Content']+test_data['Title']+test_data['Title']
df_test = df_test.map(processText)

In [22]:
pred_final = sgd.predict(tfidfv.fit_transform(hvect.fit_transform(df_test)))

In [23]:
pred_final = le.inverse_transform(pred_final)

In [25]:
to_print_arr = np.array(test_data[['Id']]).flatten()
columns = pd.Index(['Id','Predicted'])
data = np.column_stack((to_print_arr,pred_final))
df_final = pd.DataFrame(data, index=None, columns = columns)
df_final.to_csv('q1/testSet_categories.csv', sep=',', index=False)